#**Hypertic AI**

Hypertic is an open-source framework which is the fastest and easiest way to build AI agents that can use any large language model, tools, vectordb, memory, and more.

### **1. Basic Agent Setup**

Let's start with a simple agent that can answer questions and use tools.

In [ ]:
!pip install hypertic openai anthropic

In [ ]:
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
os.environ["EXA_API_KEY"] = userdata.get("EXA_API_KEY")

In [ ]:
from datetime import datetime

from hypertic.agents import Agent
from hypertic.models import OpenAIChat
from hypertic.tools import tool


# Define a custom tool
@tool
def today_date() -> str:
    """Returns the today's date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")


# Create an agent
agent = Agent(
    model=OpenAIChat(model="gpt-5.2"),
    instructions="",  # If you want you can add instructions also.
    tools=[today_date],
)

# Run the agent
response = agent.run("what is today's date?")
print(f"full response: {response}")
print(f"Final response: {response.content}")
print(f"Final tool calls: {response.tool_calls}")
print(f"Final tool outputs: {response.tool_outputs}")
print(f"Final metadata: {response.metadata}")

full response: {'content': 'Today’s date is **2026-01-16**.', 'metadata': {'model': 'gpt-5.2', 'params': {}, 'finish_reason': 'stop', 'input_tokens': 284, 'output_tokens': 29}, 'tool_calls': [{'id': 'call_fAHXfAyF26n6oUQxFKE0O3ci', 'function': {'arguments': '{}', 'name': 'today_date'}, 'type': 'function'}], 'tool_outputs': {'today_date': '2026-01-16'}}
Final response: Today’s date is **2026-01-16**.
Final tool calls: [{'id': 'call_fAHXfAyF26n6oUQxFKE0O3ci', 'function': {'arguments': '{}', 'name': 'today_date'}, 'type': 'function'}]
Final tool outputs: {'today_date': '2026-01-16'}
Final metadata: {'model': 'gpt-5.2', 'params': {}, 'finish_reason': 'stop', 'input_tokens': 284, 'output_tokens': 29}


### **2. Streaming Responses**

Get real-time responses as the model generates them.

In [ ]:
# Stream the response
for event in agent.stream("what is today's date?"):
    if event.type == "content":
        print(event.content, end="", flush=True)
    elif event.type == "tool_calls":
        print(f"Tool Calls: {event.tool_calls}")
    elif event.type == "tool_outputs":
        print(f"Tool Outputs: {event.tool_outputs}")
    elif event.type == "metadata":
        print(f"\nMetadata: {event.metadata}")

Tool Calls: [{'id': 'call_4cNfQWaOwIRBHht4QFZK5wqD', 'type': 'function', 'function': {'name': 'today_date', 'arguments': '{}'}}]
Tool Outputs: {'today_date': '2026-01-16'}
Today’s date is **2026-01-16**.
Metadata: {'model': 'gpt-5.2', 'params': {}, 'finish_reason': 'stop', 'input_tokens': 157, 'output_tokens': 16}


### **3. Async Support**

Hypertic supports both sync and async operations. Use `arun()` for async workflows.

In [ ]:
# Async Response
async def async_demo():
    response = await agent.arun("tell me a joke?")
    print("Async Response:")
    print(response.content)

    print("\nAsync Streaming:")
    async for event in agent.astream("tell me a joke about cats?"):
        if event.type == "content":
            print(event.content, end="", flush=True)


await async_demo()

Async Response:
Why don’t skeletons fight each other?

They don’t have the guts.

Async Streaming:
Why don’t cats play poker in the jungle?

Too many cheetahs.

### **4. Structured Output**

Get type-safe, structured responses using Pydantic models.

In [ ]:
from pydantic import BaseModel


# Define a response format
class Date(BaseModel):
    day: int
    month: int
    year: int


# Create an agent
structured_agent = Agent(model=OpenAIChat(model="gpt-5.2"), tools=[today_date], output_type=Date)
response = structured_agent.run("what is today's date?")
print(response.structured_output)
print(response.structured_output.day)

Date(day=16, month=1, year=2026)
16


### **5.MCP(Model Context Protocol)**

MCP configuration can indeed be defined using either JSON or TOML formats. This provides flexibility for users to choose their preferred data serialization format.

In [ ]:
from hypertic.models import Anthropic
from hypertic.tools import MCPServers


async def main():
    # MCP configuration
    config = {
        "exa": {"url": "https://mcp.exa.ai/mcp?exaApiKey=" + os.environ["EXA_API_KEY"], "transport": "streamable_http"},
    }

    # Create MCP client
    mcp_tools = await MCPServers(config).get_tools(["web_search_exa"])
    tools = [mcp_tools]
    print(f"Available Tools: {tools}")

    agent = Agent(
        model=Anthropic(model="claude-haiku-4-5"),
        instructions="",
        tools=tools,
    )

    print("\nStreaming response:")
    async for event in agent.astream("find me good restaurants in san francisco"):
        if event.type == "content":
            print(event.content, end="", flush=True)
        elif event.type == "tool_calls":
            print(f"\nTool Calls: {event.tool_calls}")
        elif event.type == "tool_outputs":
            print(f"\nTool Outputs: {event.tool_outputs}")
        elif event.type == "metadata":
            print(f"\nMetadata: {event.metadata}")


if __name__ == "__main__":
    await main()

Available Tools: [[{'name': 'web_search_exa', 'description': 'Search the web using Exa AI - performs real-time web searches and can scrape content from specific URLs. Supports configurable result counts and returns the content from the most relevant websites.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Websearch query'}, 'numResults': {'type': 'number', 'description': 'Number of search results to return (default: 8)'}, 'livecrawl': {'type': 'string', 'enum': ['fallback', 'preferred'], 'description': "Live crawl mode - 'fallback': use live crawling as backup if cached content unavailable, 'preferred': prioritize live crawling (default: 'fallback')"}, 'type': {'type': 'string', 'enum': ['auto', 'fast', 'deep'], 'description': "Search type - 'auto': balanced search (default), 'fast': quick results, 'deep': comprehensive search"}, 'contextMaxCharacters': {'type': 'number', 'description': 'Maximum characters for context string optimized for 

### **6. RAG (Retrieval Augmented Generation)**

Add knowledge to your agents using vector databases. Hypertic supports ChromaDB, Qdrant, Pinecone, MongoDB Atlas, and PgVector.

In [ ]:
from hypertic.embedders import OpenAIEmbedder
from hypertic.vectordb import ChromaDB

# Create an embedder
embedder = OpenAIEmbedder(model="text-embedding-3-small")
vector_db = ChromaDB(collection="demo", embedder=embedder, path="./chromadb")

# vector_db.add(
#     files=["data/sample.txt"],
#     metadatas=[{"source": "sample.txt", "type": "document"}]
# )

text = "My secret is I am not a vibecoder."
vector_db.add(texts=[text], metadatas=[{"source": "sample.txt", "type": "document"}])

retriever = vector_db.as_retriever(k=3)

rag_agent = Agent(model=OpenAIChat(model="gpt-5.2"), retriever=retriever)

response = rag_agent.run("What is my secret?")
print(response.content)

Your secret is: you’re **not a vibecoder**.

If you meant this as a riddle and want alternate interpretations (e.g., what “vibecoder” could imply), tell me the context and I’ll play along.


## **7. Memory & Conversation History**

Add memory to your agents so they remember previous conversations. Hypertic supports PostgreSQL, MongoDB, Redis, and in-memory storage.

In [ ]:
# Create in-memory memory (use PostgreSQL/MongoDB in production)
from hypertic.memory import InMemory

memory = InMemory()

memory_agent = Agent(model=OpenAIChat(model="gpt-5.2"), instructions="", memory=memory)

user_id = "user"
session_id = "session_1"

while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    if not user_input:
        continue
    try:
        response = memory_agent.run(user_input, session_id=session_id, user_id=user_id)
        print(f"Agent: {response.content}\n")
    except Exception as e:
        print(f"Error: {e}\n")

You: hello my name is prasad
Agent: Hello Prasad — nice to meet you. What would you like help with today?

You: what is my name
Agent: Your name is Prasad.

You: q


This notebook demonstrated the key features of Hypertic like tools,


### **Resources**
- Examples: [https://github.com/hypertic-ai/hypertic/examples](https://github.com/hypertic-ai/hypertic/blob/main/examples)
- Documentation: [https://docs.hypertic.ai](https://docs.hypertic.ai)



Happy Building! 🚀